# Shared Information

## Workflow

### Agile Methodology

*A brief description of what this actually means and how we are implementing it.*

A large part of this is using Kanbn for detailing, priotising, scheduling, assigning...etc. work ~ Very useful!

### Github Commits
**Gitmoji:** https://gitmoji.dev/
- ```Docs\``` Updates - 📄
- Task (Kanbn) Updates - 📝
- Folder/File Structure Changes - 📂
- Successfully Working at this point - ✅ (Specify how)

### Github Branching
- ```git checkout -b <branch-name>```
- ... (For 2/4/25) How do we not get in the way of each other working best!

# Pipeline

## Azure for Cloud Server

Azure can serve as the cloud infrastructure where your .NET C# project will run. You can leverage various Azure services such as:

- Azure App Services for deploying your application.
- Azure Blob Storage for storing files such as song data, user uploads, etc.
- Azure Kubernetes Service (AKS) if you're scaling with microservices.
- Azure Functions for serverless processing of specific tasks.
- Azure would provide scalability, high availability, and manageability for your application, particularly if it involves cloud-based processing or needs to be accessible from various locations.
  
Where to start:

- Set up an Azure account and explore Azure Portal.
- Try deploying a basic C# .NET Core Web API to Azure App Service.
- Learn about Azure Resource Manager (ARM) templates for infrastructure as code.


## MicroServices Architecture

Microservices would break down your application into smaller, independently deployable services. Each service would handle a specific function, which could make your project more maintainable and scalable. For example:

- A Song Service for managing song metadata and content.
- A User Service for handling user authentication, preferences and progress.

This approach allows each microservice to be developed, deployed, and scaled independently, which makes it easier to improve or replace one part of the system without affecting the rest.

Where to start:

- Build a small .NET Web API service and containerize it with Docker.
- Learn ASP.NET Core minimal APIs for lightweight microservices.
- Set up an API Gateway using Ocelot or Azure API Management.

## Flat file rest API for the songs

**Website:** https://www.geeksforgeeks.org/flat-file-database/

**Flat File:** A flat file can be a binary file or a plain text file such as CSV, txt, or TSV file. 

Where to start:

- Create a simple ASP.NET Core API that reads/writes JSON files.
- Use System.Text.Json or Newtonsoft.Json for serialization.
- Add authentication if needed with JWT or API Keys.

## Agile Methodology for Continous Improvement

- Use Kanban to manage tasks.
- Track progress with JIRA, Azure DevOps, or Trello.
- CI/CD pipelines in GitHub Actions or Azure DevOps for deployment.
- Frequent retrospectives to improve team workflow.

# Setting up

## .NET

- .Net SDK 9.0.201: https://dotnet.microsoft.com/en-us/download/dotnet/9.0

## Azure

### Creating ASP.NET app & deploying on Azure as a web app

Tutorial Page: https://learn.microsoft.com/en-us/azure/app-service/quickstart-dotnetcore?tabs=netframework48&pivots=development-environment-vscode

Tutorial Videos: https://www.youtube.com/watch?v=EqNJ9IQgODI, https://www.youtube.com/watch?v=y8Hy0uYL3hg, https://www.youtube.com/watch?v=OEdZGF1JcEI, https://www.youtube.com/watch?v=Ud5CqyMHKTg

## Connect

The next steps require:
> 1. Deloy: Deployment of the .NET into the App Service, 
> 2. Enable: enable a managed identity on app service: \
> Enable: In the App Service, go to Identity (under Settings) and turn on System-assigned Managed Identity.
> 3. Grant Acess: grant the app service access:
> Go to Azure SQL Server, under Active Directory admin, enable Entra authentication. Then go to the database and open Query Editor and run: \
> ```CREATE USER [your-app-service-name] FROM EXTERNAL PROVIDER;```\
> ```ALTER ROLE db_owner ADD MEMBER [your-app-service-name];```
> 4. Connect: connect in .NET:
> In appsettings.json - ```json
{
  "ConnectionStrings": {
    "DefaultConnection": "Server=tcp:lyrical-sql-server.database.windows.net,1433;Database=LyricalLearningDB;Encrypt=True;Authentication=Active Directory Default;"
  }
}``` \
> cmd: ```dotnet add package Azure.Identity```, ```dotnet add package Microsoft.Data.SqlClient```
> In Program.cs - ```var credential = new DefaultAzureCredential();```